In [ ]:
import gdspy
import numpy as np


In [ ]:
# The GDSII file is called a library, which contains multiple cells.
lib = gdspy.GdsLibrary(unit=1e-6, precision=1e-9)

# Geometry must be placed in cells.
cell = lib.new_cell('angle_waveguide')

tol = 0.0001 # This control how good circles/arcs really is
fontsize = 10


In [ ]:
# waveguide inputs
t_waveguide = 0.5
w_waveguides = np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
r_waveguide = 100.0
d_waveguide = 25
L_total = 5000.0
rotations_geometry = np.linspace(-85, 85, 35)*np.pi/180
rotations_material = -rotations_geometry
#-------------

# ring inputs
r_ring = 80.0
gap = 0.85
#-------------

# aux inputs
l_square = 10.0
l_taper = 3000.0
w_taper = 0.30
final_y_coordinate = 12000
ruler_precision = 100
#-------------

# doses and names
sample_number = "CCR_AngWav_005"
ebeam_brand = "EL7"
ebeam_energy = 150 # in keV
ebeam_dose = "4x200=800" # in uC/cm2
ebeam_current = 2 # in nA
field_size = 500
cut = "X"
#-------------

# aux functions
def waveguide_segment(waveguide, L, len_SC, layer):
    waveguide.segment(L, layer=layer)
    len_SC[0] = len_SC[0] + np.abs(L)
    
def waveguide_arc(waveguide, R, θi, θf, tol, len_SC, layer):
    waveguide.arc(R, θi, θf, tolerance=tol, layer=layer)
    len_SC[1] = len_SC[1] + R*np.abs(θi - θf)
#-------------


In [ ]:
for i in range(len(rotations_geometry)):
    θ = rotations_geometry[i]   
    for j in range(len(w_waveguides)):
        waveguide_layer = i
        ring_layer = i
        header_layer = i

        len_SC = np.array([0.0, 0.0])
        
        if j==0:
            x0 = i*(len(w_waveguides)+1)*d_waveguide
        else:
            x0 = x0 + d_waveguide + w_waveguides[j-1]/2 + w_waveguides[j]/2
        
        w = w_waveguides[j]
        y0 = 0

        square = gdspy.Rectangle((x0, 0), (x0+l_square, l_square), layer=waveguide_layer)
        square.translate(-l_square/2, 0)
        cell.add(square)
        
        waveguide = gdspy.Path(w_taper, (x0, 0))
        waveguide.segment(l_taper, '+y', final_width = w, layer=waveguide_layer)
        
        l_max = 1750
        waveguide.segment(l_max*np.cos((np.pi*θ)/(np.max(rotations_geometry)*2)), layer=waveguide_layer)
        
        Δd = (d_waveguide + w/2)*np.abs(np.tan(θ/2))
        
        if θ>0:
            waveguide.segment((len(w_waveguides)-j-1)*Δd, layer=waveguide_layer)
            waveguide_arc(waveguide, r_waveguide, np.pi, np.pi - θ, tol, len_SC, layer=waveguide_layer)
        elif θ<0:
            waveguide.segment(j*Δd, layer=waveguide_layer)
            waveguide_arc(waveguide, r_waveguide, 0, -θ, tol, len_SC, layer=waveguide_layer)
        
        ticks = int(np.max(np.transpose(waveguide.polygons[-1])[1])/ruler_precision)
        if (i==0 and j==0) or (i==len(rotations_geometry)-1 and j==len(w_waveguides)-1):
            for k in range(ticks):
                marker = ticks*ruler_precision-(k+1)*ruler_precision
                ruler = gdspy.Text(str(int(marker)), fontsize, layer=header_layer)
                if i==0:
                    ruler.translate(x0-65, k*ruler_precision)
                else:
                    ruler.translate(x0+35, k*ruler_precision)
                cell.add(ruler)
            
        L_ring1 = 3*L_total/5
        waveguide_segment(waveguide, L_ring1, len_SC, layer=waveguide_layer)
        
        if j==0:
            y_ring = np.max(np.transpose(waveguide.polygons[-1])[1])
            y_ring_idx = np.argmax(np.transpose(waveguide.polygons[-1])[1]) 
            x_ring = np.transpose(waveguide.polygons[-1])[0][y_ring_idx]
            if θ>0:
                ring = gdspy.Round((x_ring-(r_ring+w/2+gap)*np.cos(θ), y_ring+(r_ring+w/2+gap)*np.sin(θ)), r_ring+w/2, inner_radius=r_ring-w/2, tolerance=tol, layer=ring_layer)
            else:
                ring = gdspy.Round((x_ring-(r_ring+3*w/2+gap)*np.cos(θ), y_ring+(r_ring+3*w/2+gap)*np.sin(θ)), r_ring+w/2, inner_radius=r_ring-w/2, tolerance=tol, layer=ring_layer)
            cell.add(ring)
            
        L_ring2 = L_total/5
        waveguide_segment(waveguide, L_ring2, len_SC, layer=waveguide_layer)
        
        if j==len(w_waveguides)-1:
            y_ring = np.max(np.transpose(waveguide.polygons[-1])[1])
            y_ring_idx = np.argmax(np.transpose(waveguide.polygons[-1])[1]) 
            x_ring = np.transpose(waveguide.polygons[-1])[0][y_ring_idx]
            header1 = "R:"+str(round(r_ring, 3))+"\n"
            header2 = "G:"+str(round(gap, 3))+"\n"
            header_all = header1+header2
            header = gdspy.Text(header_all, fontsize, layer=header_layer)
            if θ>0:
                ring = gdspy.Round((x_ring+(r_ring+3*w/2+gap)*np.cos(θ), y_ring-(r_ring+3*w/2+gap)*np.sin(θ)), r_ring+w/2, inner_radius=r_ring-w/2, tolerance=tol, layer=ring_layer)
                header.translate(x_ring+(r_ring+3*w/2+gap)*np.cos(θ), y_ring-(r_ring+3*w/2+gap)*np.sin(θ)-2*r_ring+45)
            else:
                ring = gdspy.Round((x_ring+(r_ring+w/2+gap)*np.cos(θ), y_ring-(r_ring+w/2+gap)*np.sin(θ)), r_ring+w/2, inner_radius=r_ring-w/2, tolerance=tol, layer=ring_layer)
                header.translate(x_ring+(r_ring+w/2+gap)*np.cos(θ), y_ring-(r_ring+w/2+gap)*np.sin(θ)+2*r_ring-45)      
            cell.add(header)
            cell.add(ring)
        
        waveguide_segment(waveguide, L_total - L_ring1 - L_ring2, len_SC, layer=waveguide_layer)
            
        if θ>0:
            waveguide_arc(waveguide, r_waveguide, -θ, 0, tol, len_SC, layer=waveguide_layer)
        elif θ<0:
            waveguide_arc(waveguide, r_waveguide, np.pi - θ, np.pi, tol, len_SC, layer=waveguide_layer)

        waveguide.segment(1, layer=waveguide_layer)
        x_header = np.min(np.transpose(waveguide.polygons[-1])[0]) 
        y_header = np.max(np.transpose(waveguide.polygons[-1])[1]) 
            
        if j==0:
            header1 = "angle:"+str(round(θ*180/np.pi, 3))+"\n"
            header2 = "SL:"+str(round(len_SC[0], 3))+"\n"
            header3 = "CL:"+str(round(len_SC[1], 3))+"\n"
            header4 = "L:"+str(round(len_SC[0]+len_SC[1], 3))
            header_all = header1+header2+header3+header4
            header = gdspy.Text(header_all, fontsize, layer=header_layer)
            if θ>0:
                header.translate(x_header-175, y_header)
            else:
                header.translate(x_header+200, y_header+100)
            cell.add(header)
            
            if i==0:
                dx_big_header = 0
                dy_big_header = -400
                header1 = sample_number+"\n"
                header2 = "Brand:"+ebeam_brand+"\n"
                header3 = "Energy:"+str(ebeam_energy)+"keV \n"
                header4 = "Current:"+str(ebeam_current)+"nA \n"
                header5 = "Dose:"+str(ebeam_dose)+"uC/cm2 \n\n"
                header6 = "t:"+str(round(t_waveguide, 3))+"\n"
                header7 = "w:"+str(str(w_waveguides))+"\n"
                header8 = "d:"+str(round(d_waveguide, 3))+"\n"
                header_all = header1+header2+header3+header4+header5+header6+header7+header8
                header = gdspy.Text(header_all, fontsize, layer=header_layer)
                header.translate(x_header+dx_big_header, y_header+dy_big_header)
                cell.add(header)
                
                if cut == "Z" or cut == "X": 
                    # axes
                    dx_axes = 0
                    dy_axes = -300
                    material_axis = gdspy.Path(3, (x_header+dx_axes, y_header+dy_axes))
                    material_axis.segment(50, "-y", layer=header_layer)
                    material_axis.arc(3, -np.pi, -np.pi/2, layer=header_layer)
                    material_axis.segment(50, "+x", layer=header_layer)
                    cell.add(material_axis)

                    if cut == "Z":
                        material_X_label = gdspy.Text("X", fontsize, (x_header+dx_axes+50, y_header+dy_axes-50), layer=header_layer)
                        cell.add(material_X_label)

                        material_Y_label = gdspy.Text("Y", fontsize, (x_header+dx_axes, y_header+dy_axes), layer=header_layer)
                        cell.add(material_Y_label)

                        brand = gdspy.Text("Z-cut BSBS", fontsize, (x_header+dx_axes+50, y_header+dy_axes), layer=header_layer)
                        cell.add(brand)

                    if cut == "X":
                        material_X_label = gdspy.Text("Y", fontsize, (x_header+dx_axes+50, y_header+dy_axes-50), layer=header_layer)
                        cell.add(material_X_label)

                        material_Y_label = gdspy.Text("Z", fontsize, (x_header+dx_axes, y_header+dy_axes), layer=header_layer)
                        cell.add(material_Y_label)

                        brand = gdspy.Text("X-cut BSBS", fontsize, (x_header+dx_axes+50, y_header+dy_axes), layer=header_layer)
                        cell.add(brand)

            ticks = int((final_y_coordinate-y_header)/ruler_precision)
            for k in range(ticks+1):
                marker = k*ruler_precision
                ruler = gdspy.Text(str(int(marker)), fontsize, layer=header_layer)
                if θ>0:
                    ruler.translate(x_header+165, y_header+k*ruler_precision)
                else:
                    ruler.translate(x_header-65, y_header+k*ruler_precision)
                cell.add(ruler)
        
        y_ref = np.max(np.transpose(waveguide.polygons[-1])[1])
        waveguide.segment(final_y_coordinate-y_ref, final_width = w_taper, layer=waveguide_layer)

        x_square_1 = np.min(np.transpose(waveguide.polygons[-1])[0])
        x_square_2 = np.max(np.transpose(waveguide.polygons[-1])[0])
        x_square = (x_square_1+x_square_2)/2
        y_square = np.max(np.transpose(waveguide.polygons[-1])[1])
        square = gdspy.Rectangle((x_square, y_square), (x_square+l_square, y_square+l_square), layer=waveguide_layer)
        square.translate(-l_square/2, -l_square)
        cell.add(square)
        
        cell.add(waveguide)


In [ ]:
# for i in np.arange(24):
#     for j in np.arange(22):
#         write_field = gdspy.Rectangle((0, 0), (field_size, field_size), layer=255)
#         write_field.translate(i*field_size-4350+164, j*field_size-100)
#         cell.add(write_field)
        

In [ ]:
# Save the library in a file called 'first.gds'.
lib.write_gds(sample_number+'.gds')
